In [1]:
import os
import openai

from getpass import getpass
print('Enter OpenAI API key:')
openai.api_key = getpass()

os.environ['OPENAI_API_KEY']=openai.api_key
!head '{"ftune_training_data_gpt.jsonl"}'

Enter OpenAI API key:


KeyboardInterrupt: Interrupted by user

In [11]:
client = openai.OpenAI()

client.files.create(
  file=open("ftune_training_data_gpt.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-TH6dhh6OluzNJsYrbKdtQz4J', bytes=63614, created_at=1714882303, filename='ftune_training_data_gpt.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [12]:
client.fine_tuning.jobs.create(
  training_file="file-TH6dhh6OluzNJsYrbKdtQz4J", 
  model="gpt-4.1-mini"
)

FineTuningJob(id='ftjob-FkxLIdCKcVJhM7w8HiuDvAY6', created_at=1714882319, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-CZlFzbDxdf1L4vLbz7wi9rvN', result_files=[], status='validating_files', trained_tokens=None, training_file='file-TH6dhh6OluzNJsYrbKdtQz4J', validation_file=None, user_provided_suffix=None, seed=368296415, estimated_finish=None, integrations=[])

In [13]:
# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=50)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve("ftjob-FkxLIdCKcVJhM7w8HiuDvAY6")

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-FkxLIdCKcVJhM7w8HiuDvAY6", limit=10)


# Cancel a job
#client.fine_tuning.jobs.cancel("ftjob-oujejJzkr11YCLfBAk411Z5Q")

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-5G4YlQbMXHxFz0osqzS2qEvD', created_at=1714882343, level='info', message='Fine-tuning job started', object='fine_tuning.job.event', data=None, type='message'), FineTuningJobEvent(id='ftevent-XMICh7sBf9AIcC1ANnhMsO6W', created_at=1714882342, level='info', message='Files validated, moving job to queued state', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-b0MSbuzgwUWFTCUIaRgjGrwQ', created_at=1714882319, level='info', message='Validating training file: file-TH6dhh6OluzNJsYrbKdtQz4J', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3xCBVARkROACHZvA6duSL9hf', created_at=1714882319, level='info', message='Created fine-tuning job: ftjob-FkxLIdCKcVJhM7w8HiuDvAY6', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [20]:
def generate_jsonstring(finetuned_model):
  client = openai.OpenAI()
  prompt = """palafin help
SV (Closed)

need help evolving finizen into palafin with union circle
"""
  system_message = {"role" : "system", "content" : "Generate a JSON string that describes a Pokémon trade or giveaway, including sender's and receiver's Pokémon details such as the name, IV, gender, ability, hidden ability, region, language, and form of each Pokemon."}
  response = client.chat.completions.create(
      model=finetuned_model,
      messages = [
          system_message,
          {"role" : "user", "content" : prompt}
      ],
      temperature=0.7,
      max_tokens=500,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=["###"]
      )
  return response.choices[0].message.content.strip()

# Replace with your model's name
finetuned_model = "ft:gpt-3.5-turbo-0125:personal::9LNeIrae"
print(generate_jsonstring(finetuned_model))

{ "Sender's Pokemon" :  [] , "Requested Pokemon" : [] , "version" : "SV" , "action" : "closed" }
